In [64]:
import random
import math
import time
import numpy as np
import matplotlib.pyplot as plt

In [2]:
random.seed(time.time()) #To get a "really" random generator 

In [3]:
class Cell ( object) :
    
    next_ID = 0
    
    def __init__(self, state = "NORMAL", fitness = 1, mutation_rate = 0.001):
        self.state = state
        self.fitness = fitness
        self.mutation_rate = mutation_rate
        self.ID = Cell.next_ID
        self.treatment_duration = 0
        Cell.next_ID += 1
        
    def __str__(self):
        return "Cell #{0} of type : {1}".format(self.ID, self.state)
    
    def become_cancerous_or_not (self):
        proba = random.uniform(0, 0.002)
        if self.mutation_rate > proba :
            self.state = "CANCEROUS"
            self.fitness = fc
            self.mutation_rate = mu_c
    
    def apoptose (self):
        self.state = "DEAD"
        
    def treatment (self, alpha):
        self.fitness = self.fitness * alpha
        self.treatment_duration += 1

In [4]:
class Tissue ( object ) :
    
    def __init__(self, omega, alpha, fitness_n, fitness_c, mutation_rate_n, mutation_rate_c, generation_time):
        self.fitness_n = fitness_n
        self.fitness_c = fitness_c
        self.mutation_rate_n = mutation_rate_n
        self.mutation_rate_c = mutation_rate_c
        self.generation_time = generation_time
        self.omega = omega
        self.alpha = alpha
        self.pop = []
     
    def create_cell (self, state, fitness, mutation_rate):
        return ( Cell (state, fitness, mutation_rate) )
        
    
    def initial_population (self, N = 1000, prop_cancer = 0.05):
        nb_cancer_cells = round (prop_cancer *N)
        
        for i in range(N):
            if ( i < nb_cancer_cells) :
                state = "CANCEROUS"
                fitness = self.fitness_c
                mutation_rate = self.mutation_rate_c

            else :
                state = "NORMAL"
                fitness = self.fitness_n
                mutation_rate = self.mutation_rate_n
                
                
            new_cell = self.create_cell (state, fitness, mutation_rate)
            self.pop.append(new_cell)

    def stats(self):
        nb_cells = len( self.pop )
        nb_cancer_cells = 0
        nb_normal_cells = 0
        nb_dead = 0
        detection = False
        
        
        for cell in self.pop:
            if cell.state == "NORMAL":
                nb_normal_cells += 1
            elif cell.state == "CANCEROUS" :
                nb_cancer_cells += 1
            elif cell.state == "DEAD" :
                nb_dead += 1
                
            if nb_cancer_cells > self.omega * nb_cells :
                detection = True
            
        return (nb_cells, nb_normal_cells, nb_cancer_cells, detection)
            
    def get_treatment(self):
        if self.stats()[3]:
            for cell in self.pop:
                if (cell.state == "CANCEROUS") and (cell.treatment_duration < self.generation_time):
                    cell.treatment(self.alpha)
    
    def reproduce (self):
        proba = random.uniform(0, 1)
        minimum = 1
        candidat = self.pop[0]
        
        for cell in self.pop :
            #selection of an individual with a proba proportional to the fitness
            #i.e. minimize the différence between the proba and the fitness
            if ( abs( cell.fitness - proba) < minimum):
                minimum = abs( cell.fitness - proba)
                candidat = cell
        
        new_cell = self.create_cell(candidat.state, candidat.fitness, candidat.mutation_rate)

        if new_cell.state == "NORMAL":
            new_cell.become_cancerous_or_not()
        
        self.pop.append(new_cell)
    
    def apoptose(self):
        cell = random.choice(self.pop)
        self.pop.remove(cell)
        cell.apoptose()

The simulation with a while loop takes a lot of time (it ends very rarely), maybe this is due to my code. Besides, in order to have a simulation, I decided to replace the while loop by a for loop.

In [71]:
def simulation (nb_runs, omega, alpha, fn, fc, mu_n, mu_c, T, treatment = True):
    
    my_tissue = Tissue(omega , alpha, fn, fc, mu_n, mu_c, T)
    my_tissue.initial_population(N = 1000, prop_cancer = 0.1) #initial population of 1000 individus
    
    print("Initilisation : ")
    print ( "Total number of cells : {}, number of normal cells : {}, number of cancerous cells : {}".format(my_tissue.stats()[0], my_tissue.stats()[1], my_tissue.stats()[2]), "\n" ) 

    for i in range (nb_runs):
        my_tissue.reproduce()
        my_tissue.apoptose()
        
        if treatment :
            my_tissue.get_treatment()
        
    
    print ( "Statistics after", nb_runs, "runs : ")
    print ( "Total number of cells : {}, number of normal cells : {}, number of cancerous cells : {}".format(my_tissue.stats()[0], my_tissue.stats()[1], my_tissue.stats()[2]), "\n" ) 
    
    return (my_tissue.stats())

In [61]:
N = 1000 #number of runs for the simulation with the for - loop 

mu_n = 0.001 #mutation rate for normal cells
mu_c = 0.005 #mutation rate for cancerous cells

fn = 1 #fitness for normal cells
fc = 0.5 #fitness for cancerous cells

# Detection of the cancer :

omega = 0.25 #cutoff for the cancer-detection
alpha = 1/3 #treatment constant that reduces fitness for cancerous cells
T = 20 #number of generations the treatment is still efficient

In [72]:
#Without treatment :

simulation1 = simulation (N, omega, alpha, fn, fc, mu_n, mu_c, T, False)

Initilisation : 
Total number of cells : 1000, number of normal cells : 900, number of cancerous cells : 100 

Statistics after 1000 runs : 
Total number of cells : 1000, number of normal cells : 408, number of cancerous cells : 592 



In [73]:
#With treatment :

simulation2 = simulation (N, omega, alpha, fn, fc, mu_n, mu_c, T, True)

Initilisation : 
Total number of cells : 1000, number of normal cells : 900, number of cancerous cells : 100 

Statistics after 1000 runs : 
Total number of cells : 1000, number of normal cells : 457, number of cancerous cells : 543 



In [68]:
def shannon_diversity (N, n):
    p = n/N
    return (- p *  math.log(p, 2))

In [78]:
shannon_diversity (1000, simulation1[2]) #without treatment

0.44774790406761733

In [79]:
shannon_diversity (1000, simulation2[2]) #with treatment

0.4783699119937449